In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import emcee
from multiprocessing import Pool


In [ ]:
df = pd.read_csv('generated_data/cleanned_data.csv')

N0=14136417

casos = df.casos.values
mortes = df.mortes.values
mortes_vac = df.mortes_vacinados
casos_vac = df.infectados_vacinados.values
vac = df.vacinados.values
hosp = df.hospitalizados.values
hosp_vac = df.hospitalizados_vacinados.values
date = df.Date.values

casos_fit, mortes_fit, casos_vac_fit, vac_fit, hosp_fit, hosp_vac_fit,date = casos[199:],mortes[199:],casos_vac[199:],vac[199:],hosp[199:],hosp_vac[199:],date[199:]
novos_vacinados = vac

In [ ]:
# define parametros em função do inicio da vacinação
def parametros_vacina(t,h):
    phi_e = 0.7
    k_v=1/3
    p_v = 0.2
    gamma_av = 1/3.5
    gamma_sv = 1/4
    
    eps=.75
    gamma_vh=0.36
    h_v = (1/5)*h


    return (phi_e, k_v, p_v, gamma_av, gamma_sv, eps, gamma_vh, h_v)

In [ ]:
def condicoes_iniciais(e0, rho, p):
    N=14136417
    
    Rtotal0 = np.cumsum(casos)[199]/N0
    D0 = np.cumsum(mortes)[199]/N0
    
    H0 = np.cumsum(hosp_vac)[199]/N0
    I0 = np.cumsum(casos)[197:199][-1]/N0
    S0 = 1 - e0 - (1-p)*I0 - I0 - (1- rho)*Rtotal0  - D0 - H0 
    R0 = (1- rho)*Rtotal0 
    
    return [S0, e0, 0, 0, 0, (1-p)*I0, I0, 0, 0, H0, 0, R0, D0, I0, H0, 0, 0]


In [ ]:
plt.scatter(np.arange(len(mortes_fit)),(mortes_fit), s=1)

In [ ]:
def b(t,b0,b1,b2,t1,t2):
    if t<=t1:
        b=b0
    elif t2>=t>t1:
        b=b1
    elif t>t2:
        b=b2
    return b

In [ ]:
def SEIIHURD(t, y, args):
    b0, b1, b2, t1, t2, p, delta, delta_av, delta_sv, mu_s, mu_sv, mu_vh, mu_h = args
    
    beta = b(t,b0,b1,b2,t1,t2)
    
    k = 1/4
    gamma_a = 1/3.5
    gamma_s = 1/4
    gamma_h = 0.18
    h = 0.06
    tau = vac_fit[int(t)]
    phi_e, k_v, p_v, gamma_av, gamma_sv, eps, gamma_vh, h_v = parametros_vacina(t, h)
    
    S, E, V, Sv, Ev, Ia, Is, Iav, Isv, H, Hv, R, D, C, Nh, CHv, Cv = y
    
    #suscetíveis e latentes
    dSdt = -beta*S*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv) - tau/N0
    dVdt = tau/N0 - phi_e*V 
    dEdt = beta*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)*S - k*E
    dSvdt =  phi_e*V - (1-eps)*beta*Sv*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)
    dEvdt = ((1-eps)*beta*Sv*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)) - k_v*Ev
    
    #infectadosr
    dIadt = (1-p)*k*E - gamma_a*Ia
    dIsdt = p*k*E - gamma_s*Is - mu_s*Is
    dHdt = h*gamma_s*Is - gamma_h*H - mu_h*H
    
    #infectados vacinados
    dIavdt = (1-p_v)*k_v*Ev - gamma_av*Iav
    dIsvdt = p_v*k_v*Ev - gamma_sv*Isv - mu_sv*Isv
    dHvdt = h_v*gamma_sv*Isv - gamma_vh*Hv - mu_vh*Hv
    
    #recuperados
    dRdt = gamma_a*Ia + gamma_av*Iav + (1-h)*gamma_s*Is + (1-h_v)*gamma_sv*Isv + gamma_h*H + gamma_vh*Hv
    
    #Óbitos
    dDdt = mu_s*Is + mu_sv*Isv + mu_h*H + mu_vh*Hv
    
    #Curvas de crescimento
    dCdt = p*k*E
    dNHdt = h*gamma_s*Is
    dCHVdt = h_v*gamma_sv*Isv
    dCVdt = p_v*k_v*Ev

    return [dSdt, dEdt, dVdt, dSvdt, dEvdt, dIadt, dIsdt, dIavdt, dIsvdt, dHdt, dHvdt,dRdt,
             dDdt, dCdt, dNHdt, dCHVdt, dCVdt]
            

In [ ]:
def model(theta):
        # beta, t, p, delta, delta_av, delta_sv, mu_s, mu_sv, mu_vh
    b0, b1, b2, t1, t2, p, delta, delta_av, delta_sv, mu_s, mu_sv, mu_vh, mu_h, rho, e0 = theta
    params =  b0, b1, b2, t1, t2, p, delta, delta_av, delta_sv, mu_s, mu_sv, mu_vh, mu_h
    sol = solve_ivp(fun = SEIIHURD, t_span = [0, len(date)-1], args=(params,),
                    y0=condicoes_iniciais(e0, rho, p), t_eval=np.arange(len(date)-1))

    mortes_model =      (sol.y[-5,:]).astype(float)
    casos_model =       (sol.y[-4,:]).astype(float)
    hosp_model =        (sol.y[-3,:]).astype(float)
    hosp_vac_model =    (sol.y[-2,:]).astype(float)
    casos_vac_model =   (sol.y[-1,:]).astype(float)
    
    return np.r_[casos_model*N0,casos_vac_model*N0, mortes_model*N0, hosp_model*N0, hosp_vac_model*N0]


def lnlike(theta):
    y_data = np.r_[casos_fit, casos_vac_fit, np.cumsum(mortes_fit), hosp_fit, hosp_vac_fit]
    model_results = model(theta)
    return -0.5 * np.sum(((y_data[:len(model_results)] - model_results)/0.05) ** 2)

def lnprior(theta):
    b0, b1, b2, t1, t2, p, delta, delta_av, delta_sv, mu_s, mu_sv, mu_vh, mu_h, rho, e0 = theta

    if (    0.0 <   b0        < 2. 
        and 0.0 <   b1        < 2.
        and 0.0 <   b2        < 2.
        and 60  <   t1        < 80
        and 160 <   t2        < 180
        and 0.0 <   p         < 1.
        and 0.0 <   delta     < 1.
        and 0   <   delta_av  < 1.
        and 0.0 <   delta_sv  < 1.
        and 0.0 <   mu_s      < 0.8
        and 0.0 <   mu_sv     < 0.8
        and 0.0 <   mu_vh     < 0.8
        and 0   <   mu_h      < 0.8
        and 0   <   rho       < 1.
        and 0   <   e0        < 0.3
    ):
        return 0.0
    return -np.inf

def lnprob(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta)

def main(p0,nwalkers,niter,ndim,lnprob):
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 100)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter,progress=True)

    return sampler, pos, prob, state

In [ ]:
nwalkers = 40
niter = 60

In [ ]:
intervals= np.array(   [ 
                         [0, 2.], #b0
                         [0, 2.], #b1
                         [0, 2.], #b2
                         [60, 80], #t1
                         [160, 180], #t2
                         [0, 1], # p

                         [0, 1], # delta
                         [0, 1], # delta_av
                         [0, 1], #delta_sv

                         [0, 0.8], # mu_s
                         [0, 0.8], # mu_sv
                         [0, 0.8], # mu_vh
                         [0, 0.8], # mu_vh

                         [0.0,1],   #rho
                         [0.0, 0.3] #e0   
                       ])

par0 = np.random.rand(len(intervals))
initial = intervals[:,0] + par0 * (intervals[:,1] - intervals[:,0])
ndim = len(initial)
p0 = [np.array(initial) + 1e-7 * np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
sampler1, pos1, prob1, state1 = main(p0,nwalkers,niter,ndim,lnprob)

In [ ]:
samples = sampler1.flatchain
theta_max = samples[np.argmax(sampler1.flatlnprobability)]
theta_max

b0, b1, b2, t1, t2, p, delta, delta_av, delta_sv, mu_s, mu_sv, mu_vh, mu_h, rho, e0 = theta_max
params =  b0, b1, b2, t1, t2, p, delta, delta_av, delta_sv, mu_s, mu_sv, mu_vh, mu_h

sol = solve_ivp(fun = SEIIHURD, t_span = [0, len(date)-1], args=(params,),
                y0 = condicoes_iniciais(e0, rho, p), t_eval=np.arange(len(date)))


In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol.y[-4,:] # casos
#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(casos_fit), 1)

ax.scatter(date, (casos_fit[:len(date)]), color='black',zorder=3,label='Reported data')
#Plot the fitted function as a line.
ax.plot((qi*N0),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Casos',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)
# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=10)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

print((qi)[0]*N0)
print(np.cumsum(casos_fit)[0])

#Show Graph       
plt.show()

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol.y[-5,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(mortes_fit), 1)

ax.scatter(x_para_escala,  np.cumsum(mortes_fit), color='black',zorder=3,label='Reported data')
#Plot the fitted function as a line.
ax.plot(((N0*qi)),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Mortos',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=10)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)
print((qi)[0]*N0)
print(np.cumsum(mortes)[199])
#Show Graph       
plt.show()

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

    # hosp_model = (sol.y[-3,:])
    # hosp_vac_model = (sol.y[-2,:])
    # casos_vac_model = (sol.y[-1,:])

qi = sol.y[-3,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(hosp_fit), 1)

ax.scatter(x_para_escala,  np.cumsum(hosp_fit), color='black',zorder=3,label='Reported data')
#Plot the fitted function as a line.
ax.plot(((N0*qi)),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Mortos',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=10)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization

#Show Graph       
plt.show()